In [ ]:


import numpy as np
import pandas as pd 
import plotly.express as px

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.preprocessing import normalize # normalization of the matrix
import scipy
import pandas as pd

day_wise=pd.read_csv('../input/corona-virus-report/day_wise.csv')

In [ ]:
class Mish(keras.layers.Activation):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X = Activation('Mish', name="conv1_act")(X_input)
    '''

    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'


def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

keras.utils.get_custom_objects().update({'mish': Mish(mish)})

In [ ]:

data = day_wise.copy()
data.index = pd.to_datetime(data.Date)
data = data.drop('Date', axis=1)

data = pd.DataFrame({'NewCases': data.iloc[:, 4]})


In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()
    
    
    
def show_timeseries(df,yName):
    figsize = 20
    plt.figure(figsize=(figsize,figsize/2))
    plt.plot(df.index, df)
    plt.ylabel(yName)
    plt.xlabel('Datetime')
    

def show_forecasts(df_res):
    show_forecasts2(df_res, 'NewCases')
    
def show_forecasts2(df_res, yname):
    figsize = 20
    plt.figure(figsize=(figsize,figsize/2))
    plt.plot(df_res.index, df_res.y_true, color='red')
    plt.plot(df_res.index, df_res.y_pred, color='green')
    plt.ylabel(yname)
    plt.xlabel('Datetime')

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

"""
Computes MAPE
"""
def mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

"""
Computes SMAPE
"""
def symetric_mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0))) * 100

"""
Computes MAE, MSE, MAPE, SMAPE, R2
"""
def compute_metrics(df: pd.DataFrame) -> pd.DataFrame:
    y_true, y_pred = df['y_true'].values, df['y_pred'].values
    return compute_metrics_raw(y_true, y_pred)

def compute_metrics_raw(y_true: pd.Series, y_pred: pd.Series) -> pd.DataFrame:
    mae, mse, mape, smape, r2 = mean_absolute_error(y_true=y_true, y_pred=y_pred), mean_squared_error(y_true=y_true, y_pred=y_pred), mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), symetric_mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), r2_score(y_true=y_true, y_pred=y_pred)
    return pd.DataFrame.from_records([{'MAE': mae, 'MSE': mse, 'MAPE': mape, 'SMAPE': smape, 'R2': r2}], index=[0])

In [ ]:
show_timeseries(data, 'NewCases')

In [ ]:
maxlag = 21
data_orig = data.copy();

for x in range(1, maxlag):
    data[f'NewCases_{x}'] = data.NewCases.shift(x)

In [ ]:
X, y = data[data.index < '2021-01-01'].iloc[:, 1:], data[data.index < '2021-01-01'].NewCases

In [ ]:
X_train, y_train = X[X.index.month <= 3], y[X.index.month <= 3]
X_valid, y_valid = X[(X.index.month > 3) & (X.index.month <= 6)], y[(X.index.month > 3) & (X.index.month <= 6)]
X_test, y_test = X[(X.index.month > 6)], y [(X.index.month > 6)]


In [ ]:
X_train = X_train.dropna()
y_train = y_train[X_train.index]

In [ ]:
input_layer = keras.layers.Input(shape=X_train.shape[1])
x = keras.layers.Dense(32, activation='relu')(input_layer)
output_layer = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(input_layer, output_layer)
model.summary()

model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=['mae'])

In [ ]:


batch_size = 256
epochs = 300
history = model.fit(X_train.values, y_train.values, validation_data=(X_valid.values, y_valid.values), epochs=epochs, batch_size=batch_size, verbose=0)

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict(X_test.values)



In [ ]:
df_res = pd.DataFrame({'y_pred': y_pred.reshape(-1), 'y_true': y_test})

compute_metrics(df_res)


In [ ]:
show_forecasts(df_res)

Provedl jsem analyzu dat novych pripadu covid-19 v lednu-srpnu 2020. Vytrenoval jsem jednoduchou sit. Zkousel jsem ruzne zmeny oproti siti ze cviceni, nejefektivnejsi byla zmena poctu epoch a batch_size. I tak model nepredikuje presne, ale muzeme si vsimnout nekolika dobrych vlastnosti. Vytrenovany model napriklad spravne kopiruje trend poklesu novych pripadu na konci tydne stejne jako trend vzrustu na zacatku noveho tydne. Spravne taky kopiruje lehky celkovy rostouci trend.